In [1]:
stack = "TEST"
# stack = "STAGING"
# stack = "PRODUCTION"


# %load_ext autoreload
# %autoreload 2

In [ ]:
import datetime
import subprocess
import math
import httpx
import json
import boto3
import tqdm
import os
import operator
import pandas as pd
from google.cloud import tasks_v2
from dotenv import load_dotenv

load_dotenv(".env")


(
    NAME,
    TITILER_URL,
    TITILER_API_KEY,
    API_KEY,
    INFERENCE_URL,
    DB_URL,
    ASA_QUEUE,
    ASA_URL,
    SR_URL,
    ORCHESTRATOR_URL,
) = operator.itemgetter(
    "NAME",
    "TITILER_URL",
    "TITILER_API_KEY",
    "API_KEY",
    "INFERENCE_URL",
    "DB_URL",
    "ASA_QUEUE",
    "ASA_URL",
    "SR_URL",
    "ORCHESTRATOR_URL",
)(json.loads(os.getenv("STACK_SECRETS"))[stack])

print((NAME + "\n") * 10)

In [ ]:
# FROM LIST
scenes = [
    # "S1A_IW_GRDH_1SDV_20240628T232824_20240628T232853_054530_06A2EA_C912"
    # "S1A_IW_GRDH_1SDV_20241231T170923_20241231T170948_057239_070A6D_6C68",  # busy AIS
    # DARKS
    # "S1A_IW_GRDH_1SDV_20211205T052255_20211205T052320_040869_04DA4E_A2C7",
    # "S1A_IW_GRDH_1SDV_20201109T220028_20201109T220053_035177_041B4C_E413",
    # "S1A_IW_GRDH_1SDV_20230311T215500_20230311T215525_047602_05B785_FEA5",
    # "S1A_IW_GRDH_1SDV_20230319T023925_20230319T023950_047707_05BB0D_56A1",
    # "S1A_IW_GRDH_1SDV_20221018T144246_20221018T144311_045498_0570AF_04A0",
    # "S1B_IW_GRDH_1SDV_20200909T034850_20200909T034915_023293_02C3CD_F327",
    # "S1A_IW_GRDH_1SDV_20210707T215504_20210707T215529_038677_049070_19B8",
    # "S1A_IW_GRDH_1SDV_20201013T213406_20201013T213431_034783_040DAD_B3DD",
    # "S1A_IW_GRDH_1SDV_20200921T104144_20200921T104209_034456_04023C_C472",
    # "S1A_IW_GRDH_1SDV_20210405T054621_20210405T054646_037311_046559_AD20",
    # "S1A_IW_GRDH_1SDV_20210318T213402_20210318T213427_037058_045C9B_87A6",
    # "S1B_IW_GRDH_1SDV_20201006T115402_20201006T115425_023692_02D04C_FE86",
    # "S1A_IW_GRDH_1SDV_20230903T002427_20230903T002452_050156_060958_4BF4",
    # "S1A_IW_GRDH_1SDV_20220822T114150_20220822T114215_044664_055500_6576",
    # "S1A_IW_GRDH_1SDV_20211227T180107_20211227T180136_041198_04E546_45EF",
    # "S1A_IW_GRDH_1SDV_20200905T110703_20200905T110728_034222_03F9FE_64E8",
    # "S1A_IW_GRDH_1SDV_20210830T224346_20210830T224415_039465_04A9AE_D4CE",
    # "S1A_IW_GRDH_1SDV_20211210T094843_20211210T094908_040945_04DCDF_BD0B",
    # VESSEL TEST DATASET
    # "S1A_IW_GRDH_1SDV_20241112T144222_20241112T144247_056523_06EE04_7071",
    # "S1A_IW_GRDH_1SDV_20210125T004100_20210125T004125_036287_0441CB_FC3D",
    # "S1A_IW_GRDH_1SDV_20200819T062057_20200819T062126_033972_03F134_D4C1",
    # "S1A_IW_GRDH_1SDV_20201003T194034_20201003T194059_034636_040895_A19E",
    # "S1A_IW_GRDH_1SDV_20210428T092124_20210428T092152_037648_04711A_6EC7",
    # "S1A_IW_GRDH_1SDV_20201209T025943_20201209T030008_035603_042A0A_899D",
    # "S1A_IW_GRDH_1SDV_20200816T061108_20200816T061133_033928_03EF95_6194",
    # "S1A_IW_GRDH_1SDV_20201108T045217_20201108T045242_035152_041A64_69DE",
    # "S1A_IW_GRDH_1SDV_20210130T113401_20210130T113426_036367_044493_02B1",
    # "S1B_IW_GRDH_1SDV_20201114T013822_20201114T013850_024254_02E1E1_4638",
    # "S1A_IW_GRDH_1SDV_20210626T060219_20210626T060244_038507_048B4C_5AE1",
    # "S1A_IW_GRDH_1SDV_20210711T062137_20210711T062202_038726_0491D7_B424",
    # "S1A_IW_GRDH_1SDV_20201210T224256_20201210T224325_035630_042AF4_1998",
    # "S1A_IW_GRDH_1SDV_20210111T005559_20210111T005624_036083_043AB6_3173",
    # "S1A_IW_GRDH_1SDV_20200808T222606_20200808T222633_033821_03EBDD_2485",
    # "S1A_IW_GRDH_1SDV_20200805T220025_20200805T220050_033777_03EA60_3031",
    # "S1A_IW_GRDH_1SDV_20210222T213607_20210222T213632_036708_04506F_C174",
    # "S1A_IW_GRDH_1SDV_20201211T115403_20201211T115432_035638_042B31_BAD4",
    # "S1A_IW_GRDH_1SDV_20210125T004125_20210125T004148_036287_0441CB_9D3B",
    # "S1A_IW_GRDH_1SDV_20210114T102514_20210114T102542_036132_043C73_5FD7",
    # "S1A_IW_GRDH_1SDV_20200802T062802_20200802T062827_033724_03E89F_6485",
    # "S1A_IW_GRDH_1SDV_20210114T080140_20210114T080205_036131_043C63_3037",
    # "S1A_IW_GRDH_1SDV_20201223T115432_20201223T115457_035813_04313B_3F2D",
    # "S1A_IW_GRDH_1SDV_20210123T040814_20210123T040839_036260_0440DA_D0AD",
    # "S1A_IW_GRDH_1SDV_20210127T110726_20210127T110751_036322_044313_DE5C",
    # "S1A_IW_GRDH_1SDV_20200826T062739_20200826T062804_034074_03F4B5_151E",
    # "S1A_IW_GRDH_1SDV_20200826T174457_20200826T174524_034081_03F4F8_1812",
    # "S1A_IW_GRDH_1SDV_20200729T034859_20200729T034924_033664_03E6D3_93EF",
    # "S1A_IW_GRDH_1SDV_20200918T163714_20200918T163739_034415_0400BB_0649",
    # "S1A_IW_GRDH_1SDV_20210120T220140_20210120T220208_036227_043FC4_B850",
    # "S1A_IW_GRDH_1SDV_20210208T215134_20210208T215159_036504_04495A_8ACE",
    # "S1A_IW_GRDH_1SDV_20201008T140751_20201008T140816_034706_040AEE_EAD2",
    # "S1A_IW_GRDH_1SDV_20210518T163942_20210518T164007_037945_047A75_22BA",
    # "S1A_IW_GRDH_1SDV_20200911T152715_20200911T152740_034313_03FD2A_FDFB",
    # "S1A_IW_GRDH_1SDV_20201224T104941_20201224T105010_035826_0431AC_AAED",
    # "S1A_IW_GRDH_1SDV_20210308T211922_20210308T211947_036912_04578C_32C3",
    # "S1A_IW_GRDH_1SDV_20210524T185906_20210524T185931_038034_047D29_07CF",
    # "S1A_IW_GRDH_1SDV_20210216T054436_20210216T054501_036611_044D0A_F4BB",
    # "S1A_IW_GRDH_1SDV_20201208T180156_20201208T180221_035598_0429D7_BBDB",
    # "S1A_IW_GRDH_1SDV_20210409T215314_20210409T215342_037379_0467B9_03F6",
    # "S1A_IW_GRDH_1SDV_20210523T005625_20210523T005651_038008_047C68_FE94",
    # "S1A_IW_GRDH_1SDV_20201106T155808_20201106T155833_035130_04199D_4800",
    # "S1A_IW_GRDH_1SDV_20210422T133502_20210422T133531_037564_046E16_70CC",
    # "S1A_IW_GRDH_1SDV_20201116T052225_20201116T052250_035269_041E82_12BD",
    # "S1A_IW_GRDH_1SDV_20200828T061109_20200828T061134_034103_03F5BD_EBE2",
    # "S1A_IW_GRDH_1SDV_20210621T183420_20210621T183445_038442_048944_E1A2",
    # "S1A_IW_GRDH_1SDV_20210612T140003_20210612T140032_038308_048551_A942",
    # "S1A_IW_GRDH_1SDV_20200727T053553_20200727T053622_033636_03E5FE_2D39",
    # "S1A_IW_GRDH_1SDV_20210316T092832_20210316T092857_037021_045B62_8A50",
    # "S1A_IW_GRDH_1SDV_20201009T201613_20201009T201642_034724_040B9A_67CD",
    # "S1A_IW_GRDH_1SDV_20201023T051329_20201023T051354_034919_041264_545A",
    # "S1A_IW_GRDH_1SDV_20210422T183442_20210422T183507_037567_046E30_59A7",
    # "S1A_IW_GRDH_1SDV_20200829T065329_20200829T065354_034118_03F64E_29F9",
    # "S1A_IW_GRDH_1SDV_20210723T060213_20210723T060243_038901_049711_A591",
    # "S1A_IW_GRDH_1SDV_20210410T115430_20210410T115455_037388_0467FA_6BBC",
    # "S1A_IW_GRDH_1SDV_20201027T044047_20201027T044112_034977_04145C_816C",
    # "S1A_IW_GRDH_1SDV_20210624T154204_20210624T154229_038484_048A93_87BF",
    # "S1A_IW_GRDH_1SDV_20200830T071607_20200830T071636_034133_03F6D5_6592",
    # "S1A_IW_GRDH_1SDV_20210808T151907_20210808T151932_039140_049E7F_3948",
    # "S1A_IW_GRDH_1SDV_20210414T111636_20210414T111701_037445_046A11_081C",
    # "S1A_IW_GRDH_1SDV_20201209T025943_20201209T030008_035603_042A0A_899D",
    # "S1A_IW_GRDH_1SDV_20200905T214055_20200905T214120_034229_03FA38_ED1A",
    # "S1A_IW_GRDH_1SDV_20201203T020715_20201203T020740_035515_0426FB_EE3B",
    # "S1A_IW_GRDH_1SDV_20200914T155236_20200914T155301_034357_03FEB5_3FF5",
    # "S1A_IW_GRDH_1SDV_20201114T034910_20201114T034935_035239_041D79_AA81",
    # "S1A_IW_GRDH_1SDV_20200829T094838_20200829T094903_034120_03F65E_0A81",
    # "S1A_IW_GRDH_1SDV_20210419T134630_20210419T134705_037520_046C92_5C67",
    # "S1A_IW_GRDH_1SDV_20200729T095401_20200729T095430_033668_03E6EE_2611",
    # "S1A_IW_GRDH_1SDV_20201104T175743_20201104T175808_035102_0418B1_CFD4",
    # "S1A_IW_GRDH_1SDV_20210811T172200_20210811T172225_039185_04A00C_ED00",
    # "S1A_IW_GRDH_1SDV_20201107T202510_20201107T202538_035147_041A3A_6755",
    # "S1A_IW_GRDH_1SDV_20210709T160646_20210709T160711_038703_049131_D410",
    # "S1A_IW_GRDH_1SDV_20210807T215206_20210807T215231_039129_049E2F_BF10",
    # "S1A_IW_GRDH_1SDV_20200811T060641_20200811T060706_033855_03ED02_3E59",
    # "S1A_IW_GRDH_1SDV_20210715T170149_20210715T170214_038791_0493BD_B473",
    # "S1A_IW_GRDH_1SDV_20241006T172800_20241006T172825_055985_06D8A6_318D",
    # "S1A_IW_GRDH_1SDV_20240815T143321_20240815T143346_055225_06BB57_5F3C",
    # "S1A_IW_GRDH_1SDV_20240407T105050_20240407T105116_053326_06772C_6A10",
    # "S1A_IW_GRDH_1SDV_20240421T061239_20240421T061304_053528_067F25_DFC3",
    # "S1A_IW_GRDH_1SDV_20231027T051408_20231027T051438_050946_062447_37BD",
    # "S1A_IW_GRDH_1SDV_20230623T010328_20230623T010353_049106_05E7AB_FDCF",
    # "S1A_IW_GRDH_1SDV_20230909T142516_20230909T142541_050252_060C96_5CBC",
    # "S1A_IW_GRDH_1SDV_20231003T142517_20231003T142542_050602_061891_0839",
    # # RANDOM SAMPLE
    # "S1A_IW_GRDH_1SDV_20240610T074116_20240610T074141_054258_069983_8582",
    # "S1A_IW_GRDH_1SDV_20240320T214552_20240320T214621_053071_066D5C_AA4B",
    # "S1A_IW_GRDH_1SDV_20231007T153408_20231007T153433_050661_061A91_0D85",
    # "S1A_IW_GRDH_1SDV_20231014T170546_20231014T170611_050764_061E1C_E683",
    # "S1A_IW_GRDH_1SDV_20240410T175333_20240410T175358_053375_06791F_3993",
    # "S1A_IW_GRDH_1SDV_20240315T053815_20240315T053840_052988_066A24_3046",
    # "S1A_IW_GRDH_1SDV_20231215T053845_20231215T053910_051661_063CFA_DA07",
    # "S1A_IW_GRDH_1SDV_20240723T215400_20240723T215429_054894_06AF92_53E1",
    # "S1A_IW_GRDH_1SDV_20240210T202302_20240210T202327_052501_06599A_63FD",
    # "S1A_IW_GRDH_1SDV_20231116T004930_20231116T004952_051235_062E38_AC30",
    # "S1A_IW_GRDH_1SDV_20241129T052846_20241129T052911_056765_06F7AD_11F2",
    # "S1A_IW_GRDH_1SDV_20240429T224924_20240429T224953_053655_06841F_67E1",
    # "S1A_IW_GRDH_1SDV_20230813T222113_20230813T222138_049863_05FF49_5E0E",
    # "S1A_IW_GRDH_1SDV_20230706T213443_20230706T213508_049308_05EDE4_1A54",
    # "S1A_IW_GRDH_1SDV_20240701T181830_20240701T181855_054571_06A462_2F4F",
    # "S1A_IW_GRDH_1SDV_20230712T232130_20230712T232155_049397_05F09E_F6EE",
    # "S1A_IW_GRDH_1SDV_20240630T050624_20240630T050649_054548_06A38B_EB46",
    # "S1A_IW_GRDH_1SDV_20230612T160459_20230612T160524_048955_05E30F_A1AA",
    # "S1A_IW_GRDH_1SDV_20240808T081352_20240808T081417_055119_06B770_589F",
    # "S1A_IW_GRDH_1SDV_20250123T034106_20250123T034131_057566_07175D_6055",
    # "S1A_IW_GRDH_1SDV_20240330T152819_20240330T152844_053213_0672C5_9A2B",
    # "S1A_IW_GRDH_1SDV_20240123T042334_20240123T042359_052229_065064_57C5",
    # "S1A_IW_GRDH_1SDV_20230809T215036_20230809T215101_049804_05FD55_F4BA",
    # "S1A_IW_GRDH_1SDV_20240715T144246_20240715T144311_054773_06AB62_D01B",
    # "S1A_IW_GRDH_1SDV_20241219T042302_20241219T042327_057056_070335_5D0C",
    # "S1A_IW_GRDH_1SDV_20231218T025933_20231218T025958_051703_063E76_8F82",
    # "S1A_IW_GRDH_1SDV_20230716T215011_20230716T215024_049454_05F265_AFA8",
    # "S1A_IW_GRDH_1SDV_20250202T053243_20250202T053308_057713_071D31_9170",
    # "S1A_IW_GRDH_1SDV_20230103T230500_20230103T230525_046626_059699_9DBF",
    # "S1A_IW_GRDH_1SDV_20240514T113443_20240514T113508_053867_068C07_3F76",
    # "S1A_IW_GRDH_1SDV_20240316T071257_20240316T071326_053003_066ABF_5EF5",
    # "S1A_IW_GRDH_1SDV_20240311T043129_20240311T043154_052929_06681F_9D00",
    # "S1A_IW_GRDH_1SDV_20240831T155135_20240831T155200_055459_06C3DF_9125",
    # "S1A_IW_GRDH_1SDV_20241208T175008_20241208T175033_056904_06FD1C_7619",
    # "S1A_IW_GRDH_1SDV_20240331T011338_20240331T011403_053219_0672FD_425A",
    # "S1A_IW_GRDH_1SDV_20230827T071624_20230827T071653_050058_0605F0_9CA0",
    # "S1A_IW_GRDH_1SDV_20231004T140053_20231004T140118_050616_0618FE_8B38",
    # "S1A_IW_GRDH_1SDV_20230728T051318_20230728T051343_049619_05F775_2C9C",
    # "S1A_IW_GRDH_1SDV_20250103T154859_20250103T154924_057282_070C1B_74FB",
    # "S1A_IW_GRDH_1SDV_20241020T164533_20241020T164558_056188_06E0BE_D5C8",
    # "S1A_IW_GRDH_1SDV_20241127T105522_20241127T105542_056739_06F6A4_7702",
    # "S1A_IW_GRDH_1SDV_20240215T064452_20240215T064521_052566_065BBF_365D",
    # "S1A_IW_GRDH_1SDV_20230516T171220_20230516T171245_048562_05D752_22B6",
    # "S1A_IW_GRDH_1SDV_20230911T173429_20230911T173454_050283_060DA1_D009",
    # "S1A_IW_GRDH_1SDV_20240707T062802_20240707T062827_054651_06A726_4AFD",
    # "S1A_IW_GRDH_1SDV_20231211T185043_20231211T185108_051611_063B3D_2DF3",
    # "S1A_IW_GRDH_1SDV_20241216T040108_20241216T040133_057012_07016E_A6F2",
    # "S1A_IW_GRDH_1SDV_20240423T232442_20240423T232511_053567_0680B5_FD0D",
    # "S1A_IW_GRDH_1SDV_20231028T151113_20231028T151138_050967_0624FA_D481",
    # "S1A_IW_GRDH_1SDV_20230219T155039_20230219T155104_047307_05AD7F_B60D",
    # "S1A_IW_GRDH_1SDV_20240414T031519_20240414T031544_053424_067B07_8FF8",
    # "S1A_IW_GRDH_1SDV_20241222T062210_20241222T062234_057101_0704F8_AC47",
    # "S1A_IW_GRDH_1SDV_20240213T082133_20240213T082202_052537_065ACF_C705",
    # "S1A_IW_GRDH_1SDV_20240212T063811_20240212T063836_052522_065A48_7C10",
    # "S1A_IW_GRDH_1SDV_20230801T221003_20230801T221032_049688_05F994_9D2C",
    # "S1A_IW_GRDH_1SDV_20240915T032942_20240915T033016_055670_06CC3D_B036",
    # "S1A_IW_GRDH_1SDV_20240719T212441_20240719T212506_054835_06AD8B_EC92",
    # "S1A_IW_GRDH_1SDV_20230521T235425_20230521T235450_048639_05D99A_C1E2",
    # "S1A_IW_GRDH_1SDV_20230527T145037_20230527T145102_048721_05DC07_30D1",
    # "S1A_IW_GRDH_1SDV_20231212T065321_20231212T065346_051618_063B89_267B",
    # "S1A_IW_GRDH_1SDV_20230515T181428_20230515T181453_048548_05D6E6_6955",
    # "S1A_IW_GRDH_1SDV_20230501T183511_20230501T183536_048344_05D077_E956",
    # "S1A_IW_GRDH_1SDV_20241114T064041_20241114T064106_056547_06EEF8_1FD1",
    # "S1A_IW_GRDH_1SDV_20230110T111802_20230110T111827_046721_0599CC_0F57",
    # "S1A_IW_GRDH_1SDV_20230101T164847_20230101T164912_046593_059574_F6B5",
    # "S1A_IW_GRDH_1SSV_20240701T095803_20240701T095828_054566_06A43A_B908",
    # "S1A_IW_GRDH_1SDV_20240529T220320_20240529T220345_054092_0693CC_95BB",
    # "S1A_IW_GRDH_1SDV_20240907T153543_20240907T153608_055561_06C7E8_256B",
    # "S1A_IW_GRDH_1SDV_20241225T175752_20241225T175817_057152_070707_8CE3",
    # "S1A_IW_GRDH_1SDV_20230323T215226_20230323T215255_047777_05BD6A_91AB",
    # "S1A_IW_GRDH_1SDV_20230903T020820_20230903T020845_050157_06095E_7B36",
    # "S1A_IW_GRDH_1SDV_20230827T185148_20230827T185213_050065_06063B_0FF8",
    # "S1A_IW_GRDH_1SDV_20240917T062123_20240917T062148_055701_06CD6F_92DC",
    # "S1A_IW_GRDH_1SDV_20230401T231042_20230401T231107_047909_05C1CF_ED48",
    # "S1A_IW_GRDH_1SDV_20230611T041430_20230611T041455_048933_05E264_250A",
    # "S1A_IW_GRDH_1SDV_20230531T155729_20230531T155754_048780_05DDC8_7A47",
    # "S1A_IW_GRDH_1SDV_20230405T170422_20230405T170447_047964_05C3B7_7DAB",
    # "S1A_IW_GRDH_1SDV_20240206T183637_20240206T183702_052442_065793_0E82",
    # "S1A_IW_GRDH_1SDV_20230906T172238_20230906T172303_050210_060B1F_1BF6",
    # "S1A_IW_GRDH_1SDV_20240922T213443_20240922T213508_055783_06D0B7_E0AE",
    # "S1A_IW_GRDH_1SDV_20240501T213215_20240501T213240_053683_06853F_3002",
    # "S1A_IW_GRDH_1SDV_20231107T083931_20231107T083959_051108_0629DE_12E2",
    # "S1A_IW_GRDH_1SDV_20240712T160640_20240712T160705_054730_06A9E2_1A4C",
    # "S1A_IW_GRDH_1SDV_20230925T031053_20230925T031120_050478_06144E_2A63",
    # "S1A_IW_GRDH_1SDV_20240620T044058_20240620T044123_054402_069E84_50F1",
    # "S1A_IW_GRDH_1SDV_20210519T061314_20210519T061339_037953_047AB8_40A5",
    # "S1A_IW_GRDH_1SDV_20230604T074228_20230604T074253_048833_05DF5E_AA2E",
    # "S1A_IW_GRDH_1SDV_20230307T204415_20230307T204428_047543_05B57E_70D9",
    # "S1A_IW_GRDH_1SDV_20230901T155714_20230901T155742_050136_0608AA_D9F5",
    # "S1A_IW_GRDH_1SDV_20230625T090451_20230625T090516_049140_05E8BB_3B16",
    # "S1A_IW_GRDH_1SDV_20240715T081353_20240715T081418_054769_06AB41_EA18",
    # "S1A_IW_GRDH_1SDV_20230109T225136_20230109T225201_046713_059982_D45D",
    # "S1A_IW_GRDH_1SDV_20240524T021048_20240524T021113_054007_0690D1_307A",
    # "S1A_IW_GRDH_1SDV_20240904T053727_20240904T053752_055511_06C5E7_8689",
    # "S1A_IW_GRDH_1SDV_20240902T024722_20240902T024747_055480_06C4BA_6808",
    # "S1A_IW_GRDH_1SDV_20240121T140804_20240121T140829_052206_064F9C_D030",
    # "S1A_IW_GRDH_1SDV_20230629T050418_20230629T050443_049196_05EA6A_523D",
    # "S1A_IW_GRDH_1SDV_20240604T224610_20240604T224635_054180_0696C6_AD7E",
    # "S1A_IW_GRDH_1SDV_20230702T134455_20230702T134520_049245_05EBEA_E65C",
    # "S1A_IW_GRDH_1SDV_20231225T071832_20231225T071857_051808_06420B_6FAA",
]
print(len(scenes))

In [ ]:
# FROM CSV
scenes_csv = "/Users/jonathanraphael/Downloads/planet_backlog_scene_ids.csv"
scenes_pd = pd.read_csv(scenes_csv)
scenes = scenes_pd["s1_scene_id"].tolist()
print(len(scenes))

In [ ]:
# FROM CSV
scenes_csv = "/Users/jonathanraphael/Downloads/test_processed.csv"
scenes_pd = pd.read_csv(scenes_csv)
processed = scenes_pd["s1_scene_id"].tolist()
print(len(processed))

scenes = [s for s in scenes if s not in processed]
print(len(scenes))

# ORCHESTRATOR --SLOW-- 
## (via Scene Relevancy)

In [ ]:
USE_FAKE_FOOTPRINT = True
BATCH_SIZE = 100
total_records = len(scenes)
num_batches = math.ceil(total_records / BATCH_SIZE)

s3_client = boto3.client("s3")


def get_coordinates(scene_id, USE_FAKE_FOOTPRINT):
    """
    Given a scene id, retrieve the corresponding productInfo.json and return the footprint coordinates.
    """
    if USE_FAKE_FOOTPRINT:
        return [
            [
                [
                    [0.0, 0.0],
                    [0.0, 1.0],
                    [1.0, 1.0],
                    [1.0, 0.0],
                    [0.0, 0.0],
                ]
            ]
        ]
    file_key = f"GRD/{scene_id[17:21]}/{int(scene_id[21:23])}/{int(scene_id[23:25])}/IW/DV/{scene_id}/productInfo.json"
    response = s3_client.get_object(
        Bucket="sentinel-s1-l1c",
        Key=file_key,
        RequestPayer="requester",  # if required by your bucket settings
    )
    data = json.loads(response["Body"].read())
    return data["footprint"]["coordinates"]


for batch_num in range(num_batches):
    print(f"batch_num: {batch_num} of {num_batches}")
    batch = scenes[batch_num * BATCH_SIZE : (batch_num + 1) * BATCH_SIZE]
    records = [
        {
            "Sns": {
                "Message": json.dumps(
                    {
                        "id": scene_id,
                        "mode": scene_id.split("_")[1],
                        "resolution": scene_id.split("_")[2][-1],
                        "polarization": scene_id.split("_")[3][-2:],
                        "footprint": {
                            "type": "Polygon",
                            "coordinates": get_coordinates(
                                scene_id, USE_FAKE_FOOTPRINT
                            ),
                        },
                    }
                )
            }
        }
        for scene_id in tqdm.tqdm(batch)
    ]

    try:
        orchestrator_result = httpx.post(
            SR_URL,
            json={"Records": records},
            timeout=20.0,  # Adjusted timeout
            headers={"Authorization": f"Bearer {API_KEY}"},
        )
        print(f"Batch {batch_num + 1}/{num_batches}:", orchestrator_result)
    except Exception as e:
        print(f"Error processing batch {batch_num + 1}: {e}")

# ORCHESTRATOR --FAST--

In [ ]:
for scene_id in tqdm.tqdm(scenes):
    try:
        orchestrator_result = httpx.post(
            ORCHESTRATOR_URL + "/orchestrate",
            json={"scene_id": f"{scene_id}"},
            timeout=0.1,
            headers={"Authorization": f"Bearer {API_KEY}"},
        )
        print(scene_id, orchestrator_result)
    except Exception:
        # print(e)
        pass

# ASA --SLOW--

In [ ]:
import concurrent.futures


def create_task(scene_id, parent):
    # Define the payload for the task
    payload = {
        "scene_id": scene_id,
        "dry_run": False,
        "overwrite_previous": True,
    }

    # Define the task with an HTTP request
    task = {
        "http_request": {  # Specify the type of request
            "http_method": tasks_v2.HttpMethod.POST,
            "url": ASA_URL,  # The URL to send the request to
            "headers": {
                "Content-type": "application/json",
                "Authorization": f"Bearer {API_KEY}",
            },
            "body": json.dumps(
                payload
            ).encode(),  # Convert payload to JSON and encode to bytes
        }
    }

    try:
        # Create the task in the specified queue
        response = client.create_task(request={"parent": parent, "task": task})
        response
    except Exception as e:
        print(f"Error creating task for scene_id {scene_id}: {e}")


os.environ.pop("GOOGLE_APPLICATION_CREDENTIALS", None)
client = tasks_v2.CloudTasksClient()
parent = client.queue_path("cerulean-338116", "europe-west1", ASA_QUEUE)

max_workers = 100  # Example of aggressive concurrency
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    results = list(
        tqdm.tqdm(
            executor.map(lambda scene_id: create_task(scene_id, parent), scenes),
            total=len(scenes),
        )
    )

# for scene_id in tqdm.tqdm(scenes):
#     create_task(scene_id, parent)

# ASA --FAST--

In [ ]:
for scene_id in tqdm.tqdm(scenes):
    try:
        asa_result = httpx.post(
            ASA_URL,
            json={
                "scene_id": f"{scene_id}",
                "dry_run": False,
                "overwrite_previous": True,
                # "run_flags": ["INFRA"],
            },
            timeout=0.1,
            headers={"Authorization": f"Bearer {API_KEY}"},
        )
    except Exception:
        # print(e)
        pass

In [ ]:
[f"https://cerulean-ui-test-5qkjkyomta-uk.a.run.app/?scene_id={a}" for a in scenes]

https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Catalog/Examples.html

# AWS OpenDataRegistry ls and productInfo.json

## Download GeoTiffs from AWS

In [38]:
# Download GeoTiffs directly from S3 Open Data Registry using request payer and a list of Scene IDs

import boto3
import tqdm

s3_client = boto3.client("s3")
bucket_name = "sentinel-s1-l1c"

for scene_id in tqdm.tqdm(scenes):
    measurement = f"GRD/{scene_id[17:21]}/{int(scene_id[21:23])}/{int(scene_id[23:25])}/IW/DV/{scene_id}/measurement/"
    try:
        obj = s3_client.get_object(
            Bucket=bucket_name, Key=measurement + "iw-vv.tiff", RequestPayer="requester"
        )
        file_path = f"/Users/jonathanraphael/git/ceruleanserver/local/temp/outputs/_rasters/{scene_id}.tiff"
        with open(file_path, "wb") as file:
            chunks = []
            while data := obj["Body"].read(1024):
                chunks.append(data)
            file.write(b"".join(chunks))

        print(f"{scene_id} downloaded successfully!")
    except Exception:
        command = (
            f"aws s3 ls s3://{bucket_name}/{measurement} --request-payer requester"
        )
        output = subprocess.check_output(command, shell=True)
        print(output)
        print(f"{scene_id} is missing the VV file. Perhaps one of the above instead?")


print("wrapped")

In [ ]:
import asyncio
import httpx
import boto3
import json
import subprocess
import tqdm

start_date = datetime.date(2023, 1, 10)
end_date = datetime.date(2023, 1, 11)
concurrency = 25

s3_client = boto3.client("s3")
bucket_name = "sentinel-s1-l1c"
current_date = start_date
semaphore = asyncio.Semaphore(concurrency)


def extract_folders(output):
    lines = output.decode("utf-8").split("\n")
    folder_names = []
    for line in lines:
        if line.strip().startswith("PRE"):
            folder_name = line.split()[1].strip("/")
            folder_names.append(folder_name)
    return folder_names


def append_to_file(folders, filename="output.txt"):
    with open(filename, "a") as file:
        for folder in folders:
            file.write(folder + "\n")


async def send_request(semaphore, session, stack, scene_id, coordinates, API_KEY):
    async with semaphore:
        url = f"https://europe-west1-cerulean-338116.cloudfunctions.net/cerulean-cloud-{stack}-cf-sr"
        payload = {
            "Records": [
                {
                    "Sns": {
                        "Message": json.dumps(
                            {
                                "id": scene_id,
                                "mode": scene_id.split("_")[1],
                                "resolution": scene_id.split("_")[2][-1],
                                "polarization": scene_id.split("_")[3][-2:],
                                "footprint": {
                                    "type": "Polygon",
                                    "coordinates": coordinates,
                                },
                            }
                        )
                    }
                }
            ]
        }
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json",
        }
        try:
            await session.post(url, json=payload, headers=headers, timeout=60.0)
        except Exception as e:
            print(f"Error for scene_id: {scene_id}")
            print(e)


print(f"RUNNING ON: {stack}")
while current_date <= end_date:
    prefix = f"GRD/{current_date.year}/{current_date.month}/{current_date.day}/IW/DV/"
    command = f"aws s3 ls s3://{bucket_name}/{prefix} --request-payer requester"
    output = subprocess.check_output(command, shell=True)
    folders = extract_folders(output)
    id_footprint_dict = {}
    for folder in tqdm.tqdm(folders):
        file_key = f"{prefix}{folder}/productInfo.json"  # Ensure the correct path
        try:
            obj = s3_client.get_object(
                Bucket=bucket_name, Key=file_key, RequestPayer="requester"
            )
            content = obj["Body"].read()
            data = json.loads(content)
            # Extract 'id' and 'footprint' and add to dictionary
            # • store to GCP manifest
            id_footprint_dict[data["id"]] = data["footprint"]["coordinates"]
        except s3_client.exceptions.NoSuchKey:
            print(f"File not found: {file_key}")
        except Exception as e:
            print(f"Error accessing file {file_key}: {e}")

    async with httpx.AsyncClient() as session:
        tasks = [
            send_request(semaphore, session, stack, k, v, API_KEY)
            for k, v in id_footprint_dict.items()
        ]
        await asyncio.gather(*tasks)
    print("shipped")

    current_date += datetime.timedelta(days=1)
print("wrapped")

In [21]:
"""
Iterates over the dates from start_date to end_date, lists S3 folders and collects the scene ID found in it.
"""

start_date = datetime.date(2025, 1, 16)
end_date = datetime.date(2025, 1, 31)


def extract_folders(output):
    """Extract folder names from an 'aws s3 ls' command output."""
    lines = output.decode("utf-8").split("\n")
    folder_names = []
    for line in lines:
        if line.strip().startswith("PRE"):
            folder_name = line.split()[1].strip("/")
            folder_names.append(folder_name)
    return folder_names


scene_ids = []
current_date = start_date
while current_date <= end_date:
    prefix = f"GRD/{current_date.year}/{current_date.month}/{current_date.day}/IW/DV/"
    command = f"aws s3 ls s3://sentinel-s1-l1c/{prefix} --request-payer requester"
    current_date += datetime.timedelta(days=1)
    try:
        output = subprocess.check_output(command, shell=True)
    except Exception as e:
        print(f"Error listing S3 objects for prefix {prefix}: {e}")
        continue

    folders = extract_folders(output)
    scene_ids.extend(folders)

pd.DataFrame({"s1_scene_id": scene_ids}).to_csv(
    "/Users/jonathanraphael/Downloads/scene_ids.csv", index=False
)
print(f"len(scene_ids): {len(scene_ids)}")